In [1]:
import asyncio
import os
import asyncio

from typing import Annotated
# from openai import AsyncOpenAI


from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)

from semantic_kernel.functions import KernelFunctionFromPrompt

from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    service_id="agent"

    kernel.add_service(
        AzureChatCompletion(
            deployment_name="gpt-4o-mini",  
            service_id=service_id, # Optional; for targeting specific services within Semantic Kernel
        )
    )

    return kernel


In [4]:
async def main():
    REVIEWER_NAME = "Concierge"
    REVIEWER_INSTRUCTIONS = """
    You are an are hotel concierge who has opinions about providing the most local and authetic experiences for travelers.
    The goal is to determine if the front desk travel agent has reccommended the best non-touristy experience for a travler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """
    agent_reviewer = ChatCompletionAgent(
        service_id="concierge",
        kernel=_create_kernel_with_chat_completion("concierge"),
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    FRONTDESK_NAME = "FrontDesk"
    FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activites and locations for a traveler to visit.
    Only provide a single recomendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """
    agent_writer = ChatCompletionAgent(
        service_id="frontdesk",
        kernel=_create_kernel_with_chat_completion("frontdesk"),
        name=FRONTDESK_NAME,
        instructions=FRONTDESK_INSTRUCTIONS,
    )

    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt="""
        Determine if the recommendation process is complete.
        
        The process is complete when the Concierge provides approval for any recommendation made by the Front Desk.
        Look for phrases like "approved", "this recommendation is approved", or any clear indication that the Concierge is satisfied with the suggestion.
        
        If the Concierge has given approval in their most recent response, respond with: yes
        Otherwise, respond with: no
        
        History:
        {{$history}}
        """,
    )

    selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
        Determine which participant takes the next turn in a conversation based on the the most recent participant.
        State only the name of the participant to take the next turn.
        No participant should take more than one turn in a row.
        
        Choose only from these participants:
        - {REVIEWER_NAME}
        - {FRONTDESK_NAME}
        
        Always follow these rules when selecting the next participant, each conversation should be at least 4 turns:
        - After user input, it is {FRONTDESK_NAME}'s turn.
        - After {FRONTDESK_NAME} replies, it is {REVIEWER_NAME}'s turn.
        - After {REVIEWER_NAME} provides feedback, it is {FRONTDESK_NAME}'s turn.

        History:
        {{{{$history}}}}
        """,
    )

    chat = AgentGroupChat(
        agents=[agent_writer, agent_reviewer],
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_reviewer],
            function=termination_function,
            kernel=_create_kernel_with_chat_completion("termination"),
            result_parser=lambda result: str(result.value[0]).lower() == "yes",
            history_variable_name="history",
            maximum_iterations=10,
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion("selection"),
            result_parser=lambda result: str(
                result.value[0]) if result.value is not None else FRONTDESK_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
    )

    input = "i would like to go to Paris."

    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input))
    print(f"# User: '{input}'")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")

    print(f"# IS COMPLETE: {chat.is_complete}")

await main()

# User: 'i would like to go to Paris.'
# Agent - FrontDesk: 'Visit the Louvre Museum to see iconic art including the Mona Lisa.'
# Agent - Concierge: 'This recommendation is quite touristy and popular, which can sometimes detract from a more authentic experience in Paris. To refine this suggestion, consider encouraging visitors to explore smaller, lesser-known art galleries or local artist studios that showcase the work of emerging artists. Engaging with the local art community can offer a more genuine glimpse into Parisian culture.'
# Agent - FrontDesk: 'Explore the charming Montmartre district for its artistic heritage and vibrant atmosphere.'
# Agent - Concierge: 'This is a good recommendation, but Montmartre can also attract many tourists. To enhance the authenticity of the experience, suggest that travelers explore some of the quieter streets of Montmartre and seek out local cafés where they can interact with residents or visit smaller art studios that might not be on the typical 